In [2]:
using Revise

In [3]:
using ITensors
using Test
import CTMRG

[ Info: Precompiling CTMRG [c050b7f9-16a0-4967-b3f3-566d5b3f75cc] (cache misses: include_dependency fsize change (2))


In [4]:
#tagの張り替え
s1 = Index(2, "s1,m=1")
_s1 = Index(2, "_s1,m=1")
D = CTMRG.fixend(s1)
indices = inds(D)
@show D_new = replaceind(D, s1 => _s1)
#@show indices2 = findfirst(inds(D) .== new_ind)
;

D_new = replaceind(D, s1 => _s1) = ITensor ord=1
Dim 1: (dim=2|id=161|"_s1,m=1")
NDTensors.Dense{Float64, Vector{Float64}}
 2-element
 1.0
 0.0


In [5]:
σ = diagITensor([1.0, -1.0], s1 , _s1)
@show σ[s1 => 1, _s1 => 1] 
@show σ[s1 => 2, _s1 => 2] 
@show σ[s1 => 1, _s1 => 2] 
@show σ[s1 => 2, _s1 => 1] 
;

σ[s1 => 1, _s1 => 1] = 1.0
σ[s1 => 2, _s1 => 2] = -1.0
σ[s1 => 1, _s1 => 2] = 0.0
σ[s1 => 2, _s1 => 1] = 0.0


In [6]:
function sigma_tensor(ind1::Index, ind2::Index)
    return diagITensor([1.0, -1.0], ind1, ind2)
end

sigma_tensor (generic function with 1 method)

In [7]:
@testset "sigma_tensor test" begin
    s1 = Index(2, "s1,m=1")
    _s1 = Index(2, "_s1,m=1")
    σ_test = sigma_tensor(s1, _s1)
    @test σ_test[s1 => 1, _s1 => 1] == 1.0
    @test σ_test[s1 => 2, _s1 => 2] == -1.0
    @test σ_test[s1 => 1, _s1 => 2] == 0.0
    @test σ_test[s1 => 2, _s1 => 1] == 0.0
end

Test Summary:     | Pass  Total  Time
sigma_tensor test |    4      4  0.2s


Test.DefaultTestSet("sigma_tensor test", Any[], 4, false, false, true, 1.754293751565534e9, 1.754293751739831e9, false, "In[7]")

In [8]:
#1Dのtagを作成する関数
function make_1D_index_tags(s::String, N::Int)
    s_list = Vector{Index}(undef, N) #undefで初期化
    for i in 1:N
        s_list[i] = Index(2, "$s,m=$i")
    end
    return s_list
end

make_1D_index_tags (generic function with 1 method)

In [9]:
@testset "make_1D_index_tags test" begin
    s_list = make_1D_index_tags("s1", 4)

    @test length(s_list) == 4
    @test all(idx -> dim(idx) == 2, s_list)
end

Test Summary:           | Pass  Total  Time
make_1D_index_tags test |    2      2  0.1s


Test.DefaultTestSet("make_1D_index_tags test", Any[], 2, false, false, true, 1.754293752236086e9, 1.754293752287647e9, false, "In[9]")

In [10]:
#2Dのsのtagを作成する関数
L = 1
i = 2*L +1 #列
j = 2*L #行
sind_list = Array{Index}(undef, j, i) # j行 × i列
for m in 1:i #列優先でループを回す
    for n in 1:j
        sind_list[n, m] = Index(2, "s,j=$n,i=$m")
    end
end
sind_list
;

In [11]:
function make_2D_index_stags(L::Int)
    i = 2*L + 1 # 列
    j = 2*L # 行
    sind_list = Array{Index}(undef, j, i) # j行 × i列
    for m in 1:i # 列優先でループを回す
        for n in 1:j
            sind_list[n, m] = Index(2, "s,j=$n,i=$m")
        end
    end
    return sind_list
end

make_2D_index_stags (generic function with 1 method)

In [12]:
stag = make_2D_index_stags(1)

2×3 Matrix{Index}:
 (dim=2|id=242|"i=1,j=1,s")  …  (dim=2|id=490|"i=3,j=1,s")
 (dim=2|id=727|"i=1,j=2,s")     (dim=2|id=656|"i=3,j=2,s")

In [22]:
stag[:,1]

2-element Vector{Index}:
 (dim=2|id=242|"i=1,j=1,s")
 (dim=2|id=727|"i=1,j=2,s")

In [13]:
#2Dのsigmaのtagを作成する関数
function make_2D_index_σtags(L::Int)
    i = 2*L # 列
    j = 2*L + 1 # 行
    σind_list = Array{Index}(undef, j, i) # j行 × i列
    for m in 1:i 
        for n in 1:j
            σind_list[n, m] = Index(2, "σ,j=$n,i=$m")
        end
    end
    return σind_list
end 

make_2D_index_σtags (generic function with 1 method)

In [16]:
σtag = make_2D_index_σtags(L)

3×2 Matrix{Index}:
 (dim=2|id=291|"i=1,j=1,σ")  (dim=2|id=715|"i=2,j=1,σ")
 (dim=2|id=607|"i=1,j=2,σ")  (dim=2|id=435|"i=2,j=2,σ")
 (dim=2|id=294|"i=1,j=3,σ")  (dim=2|id=762|"i=2,j=3,σ")

In [21]:
σtag_up = σtag[2*L+1,:]

2-element Vector{Index}:
 (dim=2|id=294|"i=1,j=3,σ")
 (dim=2|id=762|"i=2,j=3,σ")

In [14]:
#tagの張り替えの関数 replaceindがあったので没
function change_tag(T::ITensor, old_ind::Index, new_ind::Index)
    new_T = ITensor(old_ind, new_ind)
    new_T[old_ind => 1, new_ind => 1] = 1
    new_T[old_ind => 2, new_ind => 2] = 1
    new_T[old_ind => 1, new_ind => 2] = 0
    new_T[old_ind => 2, new_ind => 1] = 0
    return new_T*T
end

change_tag (generic function with 1 method)